# Testing with NQL

In [2]:
from tools.dumps import get_filename_path, wrap_open
from tools.sparql_wrapper import SPARQLDataProviders # for the strip_namespaces() static method
                                                    # TODO: move it somewhere else
from rdflib import Graph
from rdflib.util import guess_format

wdpg_path = get_filename_path("output/wdpg_10000_common.ttl")
wdpg_format = guess_format(wdpg_path)

wdpg_graph = Graph()
wdpg_graph.parse(wdpg_path, format=wdpg_format)

#print(len(wdpg_graph))

<Graph identifier=N85d540c9388e4f6ab862834743327855 (<class 'rdflib.graph.Graph'>)>

In [3]:
s, v, o = next(wdpg_graph.triples((None, None, None)))

with wrap_open("output/wdpg_10000_common.tsv", "w") as tsv:
    for t in wdpg_graph.triples((None, None, None)):
        (s_s, v_s, o_s) = [SPARQLDataProviders.strip_namespace(elem) for elem in t]
        tsv.write("\t".join([v_s, s_s, o_s]) + "\n")

In [4]:
with wrap_open("output/wdpg_10000_common.tsv") as tsv:
    print(tsv.readline())

http://www.w3.org/ns/lemon/ontolex#lexicalForm	kgl:5z3sgveco6b7a	kgl:5z3sgveco6b7a-F0



In [5]:
# Until I get spark to work I guess I'll just use pandas

import pandas as pd

with wrap_open("output/wdpg_10000_common.tsv") as f:
    wdpg_df = pd.read_csv(f, sep='\t', names=['v', 's', 'o'])

In [6]:
# First, extract the types
rel_types = wdpg_df['v'].drop_duplicates()

In [7]:
rel_types = rel_types.to_list()

In [19]:
from tools.rdflib_extras import extract_domain_codomain

ontology_domcodom = extract_domain_codomain("../ontology.owl")
ontology_domcodom['kglprop:definition'] = ("kgl:Sense", "rdfs:Literal")

In [22]:
# Then, extract the relation types. Since we are using RDF
# we'd have to parse our ontology
# The small problem is that our ontology is quite incomplete.

# TODO: Move this somewhere else

import nql
context = nql.NeuralQueryContext()

for prop, (domain, codomain) in ontology_domcodom.items():
    print(f"relation {prop} between {domain} and {codomain}")
    context.declare_relation(prop, domain, codomain)

relation kglprop:form between kgl:Lexeme and kgl:Form
relation kglprop:grammaticalCategory between kgl:Form and kgl:GrammaticalCategory
relation kglprop:language between kgl:Lexeme and kgl:Language
relation kglprop:pos between kgl:Lexeme and kgl:POS
relation kglprop:semanticPointer between kgl:Synset and kgl:Synset
relation kglprop:sense between kgl:Lexeme and kgl:Sense
relation kglprop:synset between kgl:Lexeme and kgl:Synset
relation kglprop:definition between kgl:Sense and rdfs:Literal


In [24]:
wdpg_filtered = wdpg_df[wdpg_df['v'].isin(pd.Series(list(ontology_domcodom.keys())))]

for idx, (_, row) in enumerate(wdpg_filtered.iterrows()):
    context.load_kg([["\t".join(row)]])
    if idx % 1000 == 0:
        print(f"Processed batch no. {idx // 1000}")

#wdpg_filtered = wdpg_df.loc[wdpg_df['v'] in ontology_domcodom.keys()]

Processed batch no. 0
Processed batch no. 1
Processed batch no. 2
Processed batch no. 3
Processed batch no. 4
Processed batch no. 5
Processed batch no. 6
Processed batch no. 7
Processed batch no. 8
Processed batch no. 9
Processed batch no. 10
Processed batch no. 11
Processed batch no. 12
Processed batch no. 13
Processed batch no. 14
Processed batch no. 15
Processed batch no. 16
Processed batch no. 17
Processed batch no. 18
Processed batch no. 19
Processed batch no. 20
Processed batch no. 21
Processed batch no. 22
Processed batch no. 23
Processed batch no. 24
Processed batch no. 25
Processed batch no. 26
Processed batch no. 27
Processed batch no. 28
Processed batch no. 29
Processed batch no. 30
Processed batch no. 31
Processed batch no. 32
Processed batch no. 33
Processed batch no. 34
Processed batch no. 35
Processed batch no. 36
Processed batch no. 37
Processed batch no. 38
Processed batch no. 39
Processed batch no. 40
Processed batch no. 41
Processed batch no. 42
Processed batch no. 4

In [50]:
context.one("kgl:el6c3yc77ptlm-S0", "kgl:Sense").follow("kglprop:sense", -1).eval()

{}

In [33]:
# constancy

example_word = context.one("kgl:el6c3yc77ptlm", "kgl:Lexeme")
example_word.follow("kglprop:form").eval()

{}

In [40]:
context.one("kgl:el6c3yc77ptlm-F0", "kgl:Form").follow("kglprop:form", -1).eval()

{}

In [42]:
context.get_tf_tensor("kglprop:form")

SparseTensor(indices=tf.Tensor([[31026  6427]], shape=(1, 2), dtype=int64), values=tf.Tensor([1.], shape=(1,), dtype=float32), dense_shape=tf.Tensor([31027 13428], shape=(2,), dtype=int64))


In [43]:
forms = context.all("kgl:Form")

In [46]:
x.filtered_by

OK
